In [17]:
import pandas as pd
from models import Player, TradPlayerStats, AdvPlayerStats, Game
from sqlalchemy import and_
from data_manager import DataManager
dm = DataManager()
session = dm.get_session()

def get_and_save_player_data(player_id, save=False):
    data = session.query(
        Player,
        TradPlayerStats,
        AdvPlayerStats,
        Game
    ).join(Game, TradPlayerStats.game_id == Game.id)\
    .join(Player, TradPlayerStats.player_id == Player.id)\
    .join(AdvPlayerStats, and_(TradPlayerStats.game_id == AdvPlayerStats.game_id, 
                                TradPlayerStats.player_id == AdvPlayerStats.player_id))\
    .filter(
        (TradPlayerStats.player_id == player_id)# &
        #  (Game.season_type == "Playoffs")
    ).all()

    # Convert the query result to a DataFrame
    data_list = []
    for player, trad_stats, adv_stats, game in data:
        row = {
            'player_name': player.name,
            'player_position': player.position,
            'minutes': trad_stats.minutes,
            'points': trad_stats.pts,
            'rebounds': trad_stats.reb,
            'assists': trad_stats.ast,
            'efg': adv_stats.efg_pct,
            'fg3a': trad_stats.fg3a,
            'fg3m': trad_stats.fg3m,
            'fg3_pct': trad_stats.fg3_pct,
            'fga': trad_stats.fga,
            'fgm': trad_stats.fgm,
            'fta': trad_stats.fta,
            'ft_pct': trad_stats.ft_pct, 
            'steals': trad_stats.stl,
            'blocks': trad_stats.blk,
            'date': game.date,

        }
        data_list.append(row)
    if save:
        data_df.to_csv(f"data_pile/{player_name}.csv")
    data_df = pd.DataFrame(data_list)
    return data_df




    minutes  points  rebounds  assists  fg3m  fgm  steals  blocks        date
82     33.0      18        10        5     0    7       1       0  2024-05-14
1      41.0      27         7        6     2   11       1       2  2024-05-12
0      33.0      13         2        5     3    5       2       0  2024-05-10
5      33.0      20         4        2     3    8       1       0  2024-05-06
3      32.0       9         3        6     0    4       2       0  2024-05-04
6      46.0       7        13        6     1    3       1       1  2024-04-29
7      41.0       7         3        4     0    3       1       0  2024-04-27
2      41.0      29        15        3     0   12       1       0  2024-04-25
4      39.0      14         7        3     0    5       0       0  2024-04-22
10     32.0      12        11        7     0    5       0       0  2024-04-20
49     26.0      15         5        5     0    4       0       1  2024-04-14
39     28.0      13         4        4     1    6       1       

In [19]:
player_name = "Kentavious Caldwell-Pope"
player = session.query(Player).where(Player.name == player_name).all()[0]
game_log = get_and_save_player_data(player.id).sort_values(by="date", ascending=False)
print(game_log.head(25)[['minutes', 'points', 'rebounds', 'assists', 'fg3m', 'fgm', 'steals', 'blocks', 'date']])

    minutes  points  rebounds  assists  fg3m  fgm  steals  blocks        date
85     37.0      16         5        4     4    6       1       0  2024-05-14
1      31.0       3         1        3     1    1       1       2  2024-05-12
0      36.0      12         4        4     0    3       0       0  2024-05-10
5      27.0       4         3        0     0    2       1       0  2024-05-06
3      37.0       6         2        3     2    2       2       1  2024-05-04
6      32.0       5         3        3     1    2       1       0  2024-04-29
7      40.0      14         2        3     2    6       4       0  2024-04-27
2      35.0       5         2        1     1    2       2       0  2024-04-25
4      35.0       6         1        3     0    2       0       1  2024-04-22
9      36.0      12         2        1     4    4       3       0  2024-04-20
49     24.0      11         4        3     1    5       0       1  2024-04-14
37     28.0       5         1        1     1    2       1       